# Despliegue del Modelo

In [3]:
import json
from azureml.core import Workspace

id = open('id.json', 'r')
mi = json.load(id)

ws = Workspace.create(name="customermodel",
                      subscription_id = mi['my_id'],
                      resource_group = "model2",
                      location = "centralindia")

Deploying KeyVault with name customerkeyvaulta6443d99.
Deploying AppInsights with name customerinsights0b1fc302.
Deployed AppInsights with name customerinsights0b1fc302. Took 8.73 seconds.
Deploying StorageAccount with name customerstorage0d2fd2a3a.
Deployed KeyVault with name customerkeyvaulta6443d99. Took 26.34 seconds.
Deploying Workspace with name customermodel2.
Deployed StorageAccount with name customerstorage0d2fd2a3a. Took 41.96 seconds.
Deployed Workspace with name customermodel2. Took 41.84 seconds.


In [8]:
from azureml.core.model import Model

mname = "modelo"
registered_model = Model.register(model_path="modelo.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model modelo


In [9]:
################ score.py ###################

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)

def run(raw_data):
  try:
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    data['ModifiedDateOrdinal'] = pd.to_datetime(data['ModifiedDate']).map(pd.Timestamp.toordinal)

    df = data.drop(['ModifiedDate', 'rowguid', 'PasswordHash', 'PasswordSalt'], axis=1)

    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].fillna('None')
        df[col] = LabelEncoder().fit_transform(df[col])

    df = df.drop('ModifiedDateOrdinal', axis=1)

    result = model.predict(df).tolist()

    return json.dumps(result)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [10]:
from azureml.core.environment import Environment
virtual_env = Environment("custom_model_env")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn','numpy','joblib']) 


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) 

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\cecyp\AppData\Local\Temp\ipykernel_3584\2315670911.py:16: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [11]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
